In [175]:
import pandas as pd
import numpy as np
from ast import literal_eval

***
Contents 기반 추천 해보기
***

In [176]:
df = pd.read_csv('wondoofin.csv')
cat_df = pd.read_csv('category.csv')

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 559 non-null    int64 
 1   이름                 559 non-null    object
 2   로스터리               559 non-null    object
 3   타입                 559 non-null    object
 4   로스팅 포인트            559 non-null    object
 5   지속가능성              113 non-null    object
 6   컵 노트               559 non-null    object
 7   바디감                559 non-null    int64 
 8   신맛                 559 non-null    int64 
 9   단맛                 559 non-null    int64 
 10  쓴맛                 559 non-null    int64 
 11  커피 소개              559 non-null    object
 12  나라                 543 non-null    object
 13  로스터리 소개            559 non-null    object
 14  로스터리 주소            550 non-null    object
 15  이름.1               559 non-null    object
 16  식품의 유형             554 non-null    object
 1

In [178]:
df.columns

Index(['id', '이름', '로스터리', '타입', '로스팅 포인트', '지속가능성', '컵 노트', '바디감', '신맛', '단맛',
       '쓴맛', '커피 소개', '나라', '로스터리 소개', '로스터리 주소', '이름.1', '식품의 유형',
       '제조원 및 소재지', '유통기한', '제조일자', '내용량', '보관 방법', '원재료 및 함량',
       '제품문의 관련 주소 및 전화번호', '카페인', '로스터리ID'],
      dtype='object')

In [179]:
df.shape

(559, 26)

***
null 값 처리
- 지속가능성 : 113
- 나라 : 543
***
- 로스터리 주소 : 550
- 제조원 및 소재지 : 557/559
- 유통기한 : 557
- 보관 방법 : 545
- 식품의 유형 : 554
***

In [180]:
# 리스트 처리
df['컵 노트'] = df['컵 노트'].apply(literal_eval)
df['컵 노트'][0]

['토피', '맥아', '흑설탕']

***

In [181]:
#################################
########## 지속 가능성 ##########
#################################
for i in range(len(df)):
    if df['지속가능성'][i] is np.nan:
        df.at[i, '지속가능성'] = 0
        
df['지속가능성']

0         0
1         0
2       유기농
3      공정무역
4         0
       ... 
554       0
555       0
556       0
557       0
558       0
Name: 지속가능성, Length: 559, dtype: object

***
나라
***

In [182]:
df['나라'] = df['나라'].str.split(', ')
df['나라'][0]

['브라질', '과테말라']

In [183]:
for i in range(len(df)):
    if df['나라'][i] is np.nan:
        df.at[i, '나라'] = 0
        
df['나라']

0           [브라질, 과테말라]
1                 [세라도]
2                 [아리차]
3             [브라질, 인도]
4              [리무 볼레소]
             ...       
554                   0
555       [에티오피아, 콜롬비아]
556         [브라질, 콜롬비아]
557               [파젠다]
558    [과테말라, 콜롬비아, 인도]
Name: 나라, Length: 559, dtype: object

***
추천에 필요한 값들
1. 카페인 유무 : 카페인 칼럼
2. 블렌드/ 싱글오리진
3. 향 카테고리
4. 맛의 강도 : 쓴맛, 단맛, 바디감, 산미
***

In [184]:
features = pd.get_dummies(df, columns=['타입'], dtype=int)
features.head(2)

,id,이름,로스터리,로스팅 포인트,지속가능성,컵 노트,바디감,신맛,단맛,쓴맛,...,제조일자,내용량,보관 방법,원재료 및 함량,제품문의 관련 주소 및 전화번호,카페인,로스터리ID,타입_디카페인,타입_블렌드,타입_싱글오리진
0,43,데일리스윗,언더프레셔,미디엄다크,0,"[토피, 맥아, 흑설탕]",4,2,4,3,...,상단 표기일,140g / 1kg,"직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 보관하여 주십시오.","커피원두 100% (브라질산 60%, 과테말라산 40%)",핸디엄 고객센터 1599-2681,1,57,0,1,0
1,3896,브라질 세라도 디카페인,운조커피,미디엄다크,0,[null],4,2,4,3,...,주문 확인 후 제조,500g 1kg,"직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관",커피원두 100%,운조커피 010-4535-8819,0,67,1,0,0


In [185]:
features.columns

Index(['id', '이름', '로스터리', '로스팅 포인트', '지속가능성', '컵 노트', '바디감', '신맛', '단맛', '쓴맛',
       '커피 소개', '나라', '로스터리 소개', '로스터리 주소', '이름.1', '식품의 유형', '제조원 및 소재지',
       '유통기한', '제조일자', '내용량', '보관 방법', '원재료 및 함량', '제품문의 관련 주소 및 전화번호', '카페인',
       '로스터리ID', '타입_디카페인', '타입_블렌드', '타입_싱글오리진'],
      dtype='object')

In [186]:
features = features[['id', '나라', '로스팅 포인트', '지속가능성', '바디감', '신맛', '단맛', '쓴맛', '타입_디카페인', '타입_블렌드', '타입_싱글오리진']]
features.head(2)

,id,나라,로스팅 포인트,지속가능성,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진
0,43,"[브라질, 과테말라]",미디엄다크,0,4,2,4,3,0,1,0
1,3896,[세라도],미디엄다크,0,4,2,4,3,1,0,0


In [187]:
features['지속가능성'].unique()

array([0, '유기농', '공정무역', '직접무역'], dtype=object)

In [188]:
features['로스팅 포인트'].unique()

array(['미디엄다크', '라이트미디엄', '다크', '미디엄', '라이트'], dtype=object)

In [189]:
features = pd.get_dummies(features, columns=['지속가능성', '로스팅 포인트'], dtype=int)
features.head(2)

,id,나라,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진,지속가능성_0,지속가능성_공정무역,지속가능성_유기농,지속가능성_직접무역,로스팅 포인트_다크,로스팅 포인트_라이트,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크
0,43,"[브라질, 과테말라]",4,2,4,3,0,1,0,1,0,0,0,0,0,0,0,1
1,3896,[세라도],4,2,4,3,1,0,0,1,0,0,0,0,0,0,0,1


In [190]:
notes = pd.read_csv('category.csv')
notes = notes.loc[:, notes.columns!='컵 노트']
notes.head(2)

,id,꽃,과일,허브,달콤함,고소함,향료_풍미,초콜릿
0,43,0,0,0,1,0,1,0
1,3896,0,0,0,0,0,0,0


In [191]:
features = pd.merge(features, notes, on="id")
features.head(2)

,id,나라,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진,지속가능성_0,...,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크,꽃,과일,허브,달콤함,고소함,향료_풍미,초콜릿
0,43,"[브라질, 과테말라]",4,2,4,3,0,1,0,1,...,0,0,1,0,0,0,1,0,1,0
1,3896,[세라도],4,2,4,3,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [192]:
features = features.loc[:, features.columns!='나라']
features = features.set_index('id')
features.head(2)

,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진,지속가능성_0,지속가능성_공정무역,지속가능성_유기농,...,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크,꽃,과일,허브,달콤함,고소함,향료_풍미,초콜릿
id,,,,,,,,,,,,,,,,,,,,,
43,4,2,4,3,0,1,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
3896,4,2,4,3,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [193]:
features.to_csv('features.csv')

test = pd.read_csv('features.csv', index_col='id')
test.head()

,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진,지속가능성_0,지속가능성_공정무역,지속가능성_유기농,...,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크,꽃,과일,허브,달콤함,고소함,향료_풍미,초콜릿
id,,,,,,,,,,,,,,,,,,,,,
43,4,2,4,3,0,1,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
3896,4,2,4,3,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3458,3,4,5,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
22,4,1,3,4,0,1,0,0,1,0,...,0,0,0,0,0,0,1,1,0,1
180,5,3,5,1,0,0,1,1,0,0,...,0,0,1,0,1,0,0,0,0,1


In [204]:
test.columns

Index(['바디감', '신맛', '단맛', '쓴맛', '타입_디카페인', '타입_블렌드', '타입_싱글오리진', '지속가능성_0',
       '지속가능성_공정무역', '지속가능성_유기농', '지속가능성_직접무역', '로스팅 포인트_다크', '로스팅 포인트_라이트',
       '로스팅 포인트_라이트미디엄', '로스팅 포인트_미디엄', '로스팅 포인트_미디엄다크', '꽃', '과일', '허브',
       '달콤함', '고소함', '향료_풍미', '초콜릿'],
      dtype='object')

***
contetns based rec
***

In [194]:
test = test.loc[:, test.columns!='id']

In [195]:
embeddings = test.values
embeddings.shape

(559, 23)

In [196]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.3.2.


In [197]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity_matrix = cosine_similarity(embeddings, embeddings)
cosine_similarity_matrix.shape

(559, 559)

In [198]:
cosine_similarity_matrix[:2,:2]

array([[1.        , 0.95938348],
       [0.95938348, 1.        ]])

In [200]:
df_copy = df.copy()
#df_copy = df_copy.set_index('id')

In [201]:
def most_similar(idx, top_n=10):
    df_copy['cosine_similarity'] = cosine_similarity_matrix[idx]
    return df_copy.sort_values(by='cosine_similarity', ascending=False)[:top_n]

In [202]:
most_similar(150, top_n=5)

,id,이름,로스터리,타입,로스팅 포인트,지속가능성,컵 노트,바디감,신맛,단맛,...,제조원 및 소재지,유통기한,제조일자,내용량,보관 방법,원재료 및 함량,제품문의 관련 주소 및 전화번호,카페인,로스터리ID,cosine_similarity
150,281,백야 블렌드,오프사이트,블렌드,미디엄다크,0,"[캐슈넛, 견과류, 얼그레이, 감귤류, 달다]",3,4,4,...,경기도 파주시 조리읍 고봉로 929 2층,제조일로부터 1년,주문일로부터 3일 이내,200g / 1kg,햇빛이 들지 않는 상온에 밀폐 보관,"원두 100%(에티오피아, 콜롬비아, 브라질)",오프사이트 0507-1399-2781,1,63,1.000000
459,134,상큼한초밥 블렌딩,원더월 커피 로스터스,블렌드,미디엄다크,0,"[쌀밥, 땅콩, 아몬드, 허브, 감귤류]",3,3,3,...,"원더월 커피로스터스, 서울시 동대문구 답십리로 80 1층",제조일로 2개월,제품 표시,1kg,실온 보관,아라비카 생두 100%,코케 고객센터 070-4647-1868,1,68,0.980316
266,146,안목블렌드,보사노바 커피로스터스,블렌드,미디엄다크,0,"[감귤류, 살구, 피칸, 마카다미아, 아몬드]",4,4,5,...,"보사노바로스팅팩토리, 강원도 강릉시 경강로2660번길 7",제조일로부터 12개월,별도표시,200g / 1kg,"직사광선과 습기를 피하여 서늘한 곳에 실온 보관 하며, 개봉한 후에는 빨리 드십시오.",커피원두 100%,코케 고객센터 070-4647-1868,1,36,0.971004
500,60,메이데이,센터 커피,블렌드,미디엄다크,0,"[과일, 쥬시, 카라멜]",3,4,5,...,"(주)클라우드 핑크, 서울 성동구 서울숲2길 28-11",제조일로부터 1년,후면별도표기,210g / 500g,실온에서 직사광선을 피해 서늘한 곳에 보관하세요.,"커피 100%(과테말라 40%, 에티오피아 30%, 콜롬비아 30%)",코케 고객센터 070-4647-1868,1,42,0.963624
84,1298,그린 블렌딩 커피,링크 커피 로스터스,블렌드,미디엄다크,0,"[바닐라, 호두, 황설탕, 건포도]",4,3,4,...,주식회사 링크에프엔비 (전남 장성군 나노산단로 172 2동 1544-3162),NaN,제품 상단 별도 표기,200g / 1kg,NaN,원두커피 100% (상세 함량 별도 표기),코케 고객센터 070-4647-1868,1,30,0.961538
